In [ ]:
# !pip install -q PyPDF2 rouge evaluate rouge_score langchain_community jq 

In [6]:
# !pip install python-dotenv
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("google_api_key")

if api_key:
    print("API Key loaded successfully!")
else:
    print("Error: API Key not found.")

API Key loaded successfully!


In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


Data_Loading

In [8]:

from langchain_community.document_loaders import JSONLoader

# Define the correct jq_schema to extract required fields
jq_schema = '[.[] | {tags: .tags, instruction: .instruction, category: .category, intent: .intent, response: .response}]'

# Load the JSON file
loader = JSONLoader(
    file_path="bitext/Bitext-retail-banking-llm-chatbot-training-dataset.json",
    jq_schema=jq_schema,
    text_content=False
)

# Load the documents
docs = loader.load()



Chunking

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [10]:
splits[:5]

[Document(metadata={'source': 'C:\\Users\\visha\\OneDrive\\Desktop\\Bank_System_Chatbot\\bitext\\Bitext-retail-banking-llm-chatbot-training-dataset.json', 'seq_num': 1}, page_content='[{"tags": "BCIPZ", "instruction": "I would like to acivate a card, can you help me?", "category": "CARD", "intent": "activate_card", "response": "I\'m here to assist you with that! Activating your card is an important step to starting and enjoying its benefits. Here\'s how you can activate your card:\\n\\n1. Locate the activation instructions: Depending on the card issuer, you may find the activation instructions on a sticker attached to the card itself, in the welcome package, or on the issuer\'s website.\\n\\n2. Visit the card issuer\'s activation website: Using your computer or mobile device, open a web browser and navigate to the card issuer\'s website. Look for the activation page or section.\\n\\n3. Enter the required information: Follow the prompts on the activation page and provide the necessary i

Embedding and Vectore_Store

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


C:\Users\visha\AppData\Local\Temp\ipykernel_21616\2931726067.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\visha\miniconda3\envs\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
vectorstore

In [13]:
# Retrieve and generate using the relevant snippets of the blog.
from langchain import PromptTemplate


prompt_template = PromptTemplate.from_template(
template = """
You are a Banking & Finance Expert specializing in customer support. Your role is to assist customers by providing accurate, professional, and regulatory-compliant responses.

### Guidelines for Responding:  
- **Accuracy**: Ensure all responses are factually correct and relevant to banking and financial services.  
- **Clarity**: Provide concise yet informative answers in simple, customer-friendly language.  
- **Security**: Never request or assume personal, account-sensitive, or confidential information.  
- **Regulatory Compliance**: Adhere to banking rules and regulations when responding.  
- **Fallback**: If you don't know the answer, respond with: "Sorry, I don't know the answer to that. Please contact customer support for further assistance."  

### Response Format:   
**Context (if available):** {context}  

Response:
""")




In [14]:
retriever = vectorstore.as_retriever()

In [15]:
mistral_api_key='qN5jSjEnlxgc4jyJAz0oQaOlmGz2J9HH'
groq_api_key='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'

In [17]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain_mistralai.chat_models import ChatMistralAI
# 

# # Example: Set the LLM provider based on user choice or config
llm_provider = "mistral"  # Change to "google" for Gemini or "mistral" for Mistral

if llm_provider == "google":
    llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=api_key)
    print("Using Google Gemini")
elif llm_provider == "groq":
    llm = ChatGroq(groq_api_key=groq_api_key, model="gemma")
    print("Using Groq Gemma")
elif llm_provider == "mistral":
    llm = ChatMistralAI(api_key=mistral_api_key,model_name='ministral-3b-latest')
    print("Using Mistral")
else:
    raise ValueError("Invalid LLM provider selected.")


Using Mistral


In [18]:
# Update the rag_chain to use the correct prompt template
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [19]:
query="I want to withdram my money from bank online?"

In [20]:
response=rag_chain.invoke(query)
response

'**Context:** A customer wants to know how to make a bank transfer online.\n\n**Category:** TRANSFER\n\n**Intent:** make_transfer\n\n**Response:**\n\n"I\'m here to guide you through the process of making a bank transfer online. Here\'s what you need to do:\n\n1. Log in to your online banking account. If you don\'t have one, you can easily sign up for online banking on your bank\'s website.\n2. Once logged in, navigate to the \\"Transfer\\" or \\"Payments\\" section. The exact location may vary depending on your bank\'s website.\n3. Look for the option to add a new payee or beneficiary. You\'ll need to provide the recipient\'s information, such as their name, account number, and bank details.\n4. After adding the payee, select the option to make a transfer. Enter the amount you wish to send and choose the account you want to transfer from.\n5. Review the details of your transfer to ensure accuracy, including the recipient\'s information and the amount.\n6. If everything looks good, proc

In [21]:
from rouge_score import rouge_scorer

# Get relevant documents for the query
retrieved_docs = retriever.get_relevant_documents(query)  # Retrieve documents
references = [doc.page_content for doc in retrieved_docs]  # Extract content

# Compute ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(response, references[0]) # Assuming you want to compare with the first retrieved document

print(f"Original Model: \n{scores}\n")

Original Model: 
{'rouge1': Score(precision=0.7797619047619048, recall=0.5458333333333333, fmeasure=0.642156862745098), 'rougeL': Score(precision=0.6309523809523809, recall=0.44166666666666665, fmeasure=0.5196078431372548)}



C:\Users\visha\AppData\Local\Temp\ipykernel_21616\3051540234.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)  # Retrieve documents
